In [ ]:
from tinyagents import chainable, respond

@chainable
class Agent:
    name: str = "main_agent"

    def __init__(self):
        pass

    def run(self, input: str):
        return "Hello! I was provided with some tool outputs: " + str(input)
    
    def output_handler(self, output):
        return respond(output)

@chainable
class Tool1:
    """ Some tool """
    name: str = "tool1"

    def run(self, input):
        return "Tool 1 has been triggered."

@chainable
class Tool2:
    name: str = "tool2"
    
    """ Some other tool """
    def __init__(self, phrase: str):
        self.phrase = phrase

    def run(self, input):
        return f"Tool 2 has been {self.phrase}."

In [2]:
branch1 = Tool1() / Tool2(phrase="executed")

def router(x):
    if x.startswith("blue"):
        return Tool1.name
    
    return Tool2.name
    

branch1.bind_router(router)

ConditionalBranch(tool1 | tool2)

In [3]:
graph = branch1 | Agent()
str(graph)

'ConditionalBranch(tool1 | tool2) -> main_agent'

In [4]:
# Run without using Ray
runner = graph.compile()
runner.invoke("blue!")


 > Running node: conditional_branch_tool1_tool2

	Input: blue!

	Output (conditional_branch_tool1_tool2): {
  "content": "Tool 1 has been triggered.",
  "action": null,
  "ref": null
}

 > Running node: main_agent

	Input: Tool 1 has been triggered.

	Output (main_agent): {
  "content": "Hello! I was provided with some tool outputs: Tool 1 has been triggered.",
  "action": "respond",
  "ref": null
}


'Hello! I was provided with some tool outputs: Tool 1 has been triggered.'

In [5]:
runner.invoke("yellow!")


 > Running node: conditional_branch_tool1_tool2

	Input: yellow!

	Output (conditional_branch_tool1_tool2): {
  "content": "Tool 2 has been executed.",
  "action": null,
  "ref": null
}

 > Running node: main_agent

	Input: Tool 2 has been executed.

	Output (main_agent): {
  "content": "Hello! I was provided with some tool outputs: Tool 2 has been executed.",
  "action": "respond",
  "ref": null
}


'Hello! I was provided with some tool outputs: Tool 2 has been executed.'

In [6]:
# Run using Ray
runner = graph.compile(
    use_ray=True,
    ray_options={
        "tool1": {
            "num_replicas": 1
        },
        "tool2": {
            "num_replicas": 2,
            # "placement_group_strategy": ... 
        },
        "runner": {
            "num_replicas": 2
        }
    }
)
runner

2024-07-17 20:08:24,973	WARNING api.py:432 -- The default value for `max_ongoing_requests` has changed from 100 to 5 in Ray 2.32.0.
2024-07-17 20:08:24,974	WARNING api.py:432 -- The default value for `max_ongoing_requests` has changed from 100 to 5 in Ray 2.32.0.
2024-07-17 20:08:24,975	WARNING api.py:432 -- The default value for `max_ongoing_requests` has changed from 100 to 5 in Ray 2.32.0.


In [7]:
from ray import serve

# we can now serve it using Ray Serve

app = serve.run(runner, name="MyApp")

2024-07-17 20:08:27,175	INFO worker.py:1779 -- Started a local Ray instance. View the dashboard at http://127.0.0.1:8266 
2024-07-17 20:08:29,022	INFO handle.py:126 -- Created DeploymentHandle 'we2seygw' for Deployment(name='tool1', app='MyApp').
2024-07-17 20:08:29,024	INFO handle.py:126 -- Created DeploymentHandle 'cuugga1o' for Deployment(name='tool2', app='MyApp').
2024-07-17 20:08:29,024	INFO handle.py:126 -- Created DeploymentHandle '3fupoifv' for Deployment(name='main_agent', app='MyApp').
2024-07-17 20:08:29,025	INFO handle.py:126 -- Created DeploymentHandle '0bfpml64' for Deployment(name='tool1', app='MyApp').
2024-07-17 20:08:29,025	INFO handle.py:126 -- Created DeploymentHandle 'q53hx4in' for Deployment(name='tool2', app='MyApp').
2024-07-17 20:08:29,025	INFO handle.py:126 -- Created DeploymentHandle 'rnmhl0rv' for Deployment(name='main_agent', app='MyApp').
2024-07-17 20:08:29,027	INFO handle.py:126 -- Created DeploymentHandle 'd7wpytme' for Deployment(name='runner', app='M

(ServeReplica:MyApp:tool1 pid=5163) INFO 2024-07-17 20:10:30,877 MyApp_tool1 fkxjnqfq 7361501c-dc0b-48cc-a22e-edac8751d779 replica.py:373 - INVOKE OK 2.0ms
(ServeReplica:MyApp:runner pid=5167) INFO 2024-07-17 20:10:30,887 MyApp_runner rguvwsqb 7361501c-dc0b-48cc-a22e-edac8751d779 replica.py:373 - AINVOKE OK 45.3ms
(ServeReplica:MyApp:main_agent pid=5166) INFO 2024-07-17 20:10:30,886 MyApp_main_agent w6wu8wuk 7361501c-dc0b-48cc-a22e-edac8751d779 replica.py:373 - AINVOKE OK 2.2ms


(ServeReplica:MyApp:runner pid=5167) 
(ServeReplica:MyApp:runner pid=5167)  > Running node: conditional_branch_tool1_tool2
(ServeReplica:MyApp:runner pid=5167) 
(ServeReplica:MyApp:runner pid=5167) 	Input: blue!
(ServeReplica:MyApp:runner pid=5167) 
(ServeReplica:MyApp:runner pid=5167) 	Output (conditional_branch_tool1_tool2): {
(ServeReplica:MyApp:runner pid=5167)   "content": "Tool 1 has been triggered.",
(ServeReplica:MyApp:runner pid=5167)   "action": null,
(ServeReplica:MyApp:runner pid=5167)   "ref": null
(ServeReplica:MyApp:runner pid=5167) }
(ServeReplica:MyApp:main_agent pid=5166) 
(ServeReplica:MyApp:main_agent pid=5166)  > Running node: main_agent
(ServeReplica:MyApp:main_agent pid=5166) 
(ServeReplica:MyApp:main_agent pid=5166) 	Input: Tool 1 has been triggered.
(ServeReplica:MyApp:main_agent pid=5166) 
(ServeReplica:MyApp:main_agent pid=5166) 	Output (main_agent): {
(ServeReplica:MyApp:main_agent pid=5166)   "content": "Hello! I was provided with some tool outputs: Tool 1 

(ServeReplica:MyApp:tool1 pid=5163) INFO 2024-07-17 20:10:33,304 MyApp_tool1 fkxjnqfq 1b463dcf-6bc4-4883-9f98-a29c70e9bd27 /invoke replica.py:373 - INVOKE OK 1.7ms
(ServeReplica:MyApp:runner pid=5167) INFO 2024-07-17 20:10:33,314 MyApp_runner rguvwsqb 1b463dcf-6bc4-4883-9f98-a29c70e9bd27 /invoke replica.py:373 - __CALL__ OK 23.2ms
(ServeReplica:MyApp:main_agent pid=5166) INFO 2024-07-17 20:10:33,312 MyApp_main_agent w6wu8wuk 1b463dcf-6bc4-4883-9f98-a29c70e9bd27 /invoke replica.py:373 - AINVOKE OK 2.2ms


In [12]:
!serve status

proxies:
  6f0d1b1ef00348e85c1f0ddae8575304cb220b0be8ab9e1830a73ea7: HEALTHY
applications:
  default:
    status: RUNNING
    message: ''
    last_deployed_time_s: 1721160715.955894
    deployments:
      legal_researcher:
        status: HEALTHY
        status_trigger: CONFIG_UPDATE_COMPLETED
        replica_states:
          RUNNING: 1
        message: ''
      search_tool:
        status: HEALTHY
        status_trigger: CONFIG_UPDATE_COMPLETED
        replica_states:
          RUNNING: 1
        message: ''
      runner:
        status: HEALTHY
        status_trigger: CONFIG_UPDATE_COMPLETED
        replica_states:
          RUNNING: 1
        message: ''
target_capacity: null


In [13]:
result = await app.ainvoke.remote("blue!")
result

2024-07-17 20:10:30,807	INFO handle.py:126 -- Created DeploymentHandle '7t4r9i1x' for Deployment(name='runner', app='MyApp').
2024-07-17 20:10:30,816	INFO pow_2_scheduler.py:260 -- Got updated replicas for Deployment(name='runner', app='MyApp'): {'hm4hmxf0', 'rguvwsqb'}.


'Hello! I was provided with some tool outputs: Tool 1 has been triggered.'

In [14]:
# we can also interact with the application via REST

import requests

response = requests.post("http://localhost:8000/invoke", data="blue")

print(response.text)

Hello! I was provided with some tool outputs: Tool 1 has been triggered.
